In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("otp-ffb").getOrCreate();

In [7]:
df = spark.read.csv('Bovespa.csv', inferSchema=True, header=True)

In [8]:
df.show(3)

+-------+--------+-------+------+----------+-------+----+------+--------+----+----+----+----+-----+------+------+------+----------+
|TypeReg|    Date|BDICode|Codneg|MarketType|Company|Spec|Prazot|Currency|Open| Max| Min| Med|Close|Preofc|Preofv|Totneg|    Quatot|
+-------+--------+-------+------+----------+-------+----+------+--------+----+----+----+----+-----+------+------+------+----------+
|      1|20010102|      2| ACES3|        10|ACESITA|ON *|      |    R$  |0.85| 0.9|0.85|0.86| 0.86|  0.86|  0.87|    24|  49500000|
|      1|20010102|     96|ACES3F|        20|ACESITA|ON *|      |    R$  |0.89|0.89|0.88|0.88| 0.88|  0.81|   0.9|     2|     40633|
|      1|20010102|      2| ACES4|        10|ACESITA|PN *|      |    R$  |1.01|1.08|1.01|1.05| 1.06|  1.05|  1.06|   294|2715100000|
+-------+--------+-------+------+----------+-------+----+------+--------+----+----+----+----+-----+------+------+------+----------+
only showing top 3 rows



In [62]:
dfGbSum = df.groupBy(df.Date).sum()

In [63]:
dfGbSum.show(5)

+--------+------------+-----------+------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------+------------+
|    Date|sum(TypeReg)|  sum(Date)|sum(BDICode)|sum(MarketType)|         sum(Open)|          sum(Max)|          sum(Min)|          sum(Med)|        sum(Close)|       sum(Preofc)|       sum(Preofv)|sum(Totneg)| sum(Quatot)|
+--------+------------+-----------+------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------+------------+
|20011127|        1210|24213463670|       62200|          29070| 50535.26000000001| 51128.23999999999| 49998.44000000003|50502.079999999965|          50624.08| 29438.13999999998|26229.700000000063|      76004|158022506246|
|20020325|         528|10570731600|       26684|          11814| 26158.22999999998|26322.389999999996| 25893


[Stage 33:=====================>                                    (3 + 5) / 8]



In [12]:
dfGbSum.orderBy(dfGbSum.Date).show(5)

+--------+------------+-----------+------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------+------------+
|    Date|sum(TypeReg)|  sum(Date)|sum(BDICode)|sum(MarketType)|         sum(Open)|          sum(Max)|          sum(Min)|          sum(Med)|        sum(Close)|       sum(Preofc)|       sum(Preofv)|sum(Totneg)| sum(Quatot)|
+--------+------------+-----------+------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------+------------+
|20010102|        1028|20570384856|       51436|          23524|436609.86000000045|437197.28000000044| 429982.3400000005|433603.68000000063|430511.00000000006| 42522.72000000002|408490.34000000026|      42402| 68750347096|
|20010103|        1274|25492871222|       66564|          29808|492028.32000000036| 498972.6600000001| 49163


[Stage 8:===============================================>       (171 + 8) / 200]



In [13]:
dfGbSumOpt = df.select([df.Date, df.Close])

In [14]:
dfGbSumOpt = dfGbSumOpt.groupBy(dfGbSumOpt.Date).sum()

In [15]:
dfGbSumOpt.show(5)

+--------+-----------+------------------+
|    Date|  sum(Date)|        sum(Close)|
+--------+-----------+------------------+
|20011127|24213463670|          50624.08|
|20020325|10570731600|26018.640000000018|
|20020523| 9549789471|22904.879999999994|
|20021126|10270837638|22460.429999999993|
|20090213|17398124458| 44053.75000000002|
+--------+-----------+------------------+
only showing top 5 rows




[Stage 9:======================>                                    (3 + 5) / 8]



In [17]:
dfGbSumOpt.explain("cost")

== Optimized Logical Plan ==
Aggregate [Date#69], [Date#69, sum(cast(Date#69 as bigint)) AS sum(Date)#425L, sum(Close#81) AS sum(Close)#426], Statistics(sizeInBytes=86.4 MiB)
+- Project [Date#69, Close#81], Statistics(sizeInBytes=61.7 MiB)
   +- Relation[TypeReg#68,Date#69,BDICode#70,Codneg#71,MarketType#72,Company#73,Spec#74,Prazot#75,Currency#76,Open#77,Max#78,Min#79,Med#80,Close#81,Preofc#82,Preofv#83,Totneg#84,Quatot#85L] csv, Statistics(sizeInBytes=592.4 MiB)

== Physical Plan ==
*(2) HashAggregate(keys=[Date#69], functions=[sum(cast(Date#69 as bigint)), sum(Close#81)], output=[Date#69, sum(Date)#425L, sum(Close)#426])
+- Exchange hashpartitioning(Date#69, 200), ENSURE_REQUIREMENTS, [id=#137]
   +- *(1) HashAggregate(keys=[Date#69], functions=[partial_sum(cast(Date#69 as bigint)), partial_sum(Close#81)], output=[Date#69, sum#441L, sum#442])
      +- FileScan csv [Date#69,Close#81] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/Bovespa.c

In [19]:
dfGbSumOpt.rdd.getNumPartitions()

200

In [24]:
dfGbSumOpt.repartition(50)

DataFrame[Date: int, sum(Date): bigint, sum(sum(Date)): bigint, sum(sum(Close)): double]

In [25]:
dfGbSumOpt = dfGbSumOpt.groupBy(dfGbSumOpt.Date).sum()

In [26]:
dfGbSumOpt.show(5)

+--------+---------+--------------+-------------------+--------------------+
|    Date|sum(Date)|sum(sum(Date))|sum(sum(sum(Date)))|sum(sum(sum(Close)))|
+--------+---------+--------------+-------------------+--------------------+
|20011127| 20011127|      20011127|        24213463670|            50624.08|
|20020325| 20020325|      20020325|        10570731600|  26018.640000000018|
|20020523| 20020523|      20020523|         9549789471|  22904.879999999994|
|20021126| 20021126|      20021126|        10270837638|  22460.429999999993|
|20090213| 20090213|      20090213|        17398124458|   44053.75000000002|
+--------+---------+--------------+-------------------+--------------------+
only showing top 5 rows



In [30]:
# UDF (User Defined Functions):
def uppercase_converter(record):
  if len(record) > 5:
    return record.upper()
  else:
    return record.lower()

In [38]:
from pyspark.sql.functions import when,col, udf


22/08/12 15:28:28 WARN SimpleFunctionRegistry: The function upper replaced a previously registered function.


<function __main__.uppercase_converter(record)>

In [39]:
convertUDF = udf(lambda z: uppercase_converter(z))

In [58]:
dfUpper = df.withColumn ( 'Upper', convertUDF( df.Company  ) ) 

In [59]:
dfUpper.count()


[Stage 31:>                                                         (0 + 8) / 8]



6571472

In [61]:
df.groupBy(df.Company).sum()

DataFrame[Company: string, sum(TypeReg): bigint, sum(Date): bigint, sum(BDICode): bigint, sum(MarketType): bigint, sum(Open): double, sum(Max): double, sum(Min): double, sum(Med): double, sum(Close): double, sum(Preofc): double, sum(Preofv): double, sum(Totneg): bigint, sum(Quatot): bigint]

In [51]:
from pyspark.sql import SQLContext

In [52]:
df.createOrReplaceTempView("bovespa")

In [54]:

sql = spark.sql("select count(*) from bovespa where `Company` = 'ACESITA'")

In [55]:
sql

DataFrame[count(1): bigint]

In [56]:
sql.show()

+--------+
|count(1)|
+--------+
|    7549|
+--------+




[Stage 27:=============================>                            (4 + 4) / 8]



In [60]:
from pyspark.sql.functions import  desc

In [96]:
sql = spark.sql("select `Date`, sum(`Open`), sum(`Max`), sum(`Close`) from bovespa group by `Date`")

In [97]:
sql.show(50)

22/08/12 22:02:51 WARN MemoryStore: Not enough space to cache rdd_220_2 in memory! (computed 23.9 MiB so far)
22/08/12 22:02:51 WARN MemoryStore: Not enough space to cache rdd_220_6 in memory! (computed 48.2 MiB so far)


+--------+------------------+------------------+------------------+
|    Date|         sum(Open)|          sum(Max)|        sum(Close)|
+--------+------------------+------------------+------------------+
|20011127| 50535.26000000001| 51128.23999999999|          50624.08|
|20020325| 26158.22999999998|26322.389999999996|26018.640000000018|
|20020523|22892.239999999965|23097.109999999986|22904.879999999994|
|20021126|22624.510000000006| 22730.10000000002|22460.429999999993|
|20090213|          44384.44| 44632.58999999996| 44053.75000000002|
|20090820| 51619.49999999994|51860.719999999994| 51225.18000000004|
|20090824| 54378.76999999989|55283.839999999975| 54179.54000000009|
|20100407|44965.359999999986|45321.929999999935| 45075.69000000005|
|20111107|  60293.5799999999| 61044.68999999992|60639.259999999944|
|20120719| 92057.88999999984| 94558.07999999993| 93630.90999999986|
|20120808|116913.97999999975|119263.11999999982|116728.24999999974|
|20120810| 132699.8999999999|137796.18999999997|

In [93]:
dfGbSumOpt = df.select([df.Date, df.Close, df.Open, df.Max])

In [94]:
dfGbSum = df.groupBy(df.Date).sum()

In [95]:
dfGbSum.show(50)

22/08/12 22:02:28 WARN MemoryStore: Not enough space to cache rdd_220_6 in memory! (computed 24.5 MiB so far)
22/08/12 22:02:28 WARN MemoryStore: Not enough space to cache rdd_220_2 in memory! (computed 47.2 MiB so far)


+--------+------------+-----------+------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------+------------+
|    Date|sum(TypeReg)|  sum(Date)|sum(BDICode)|sum(MarketType)|         sum(Open)|          sum(Max)|          sum(Min)|          sum(Med)|        sum(Close)|       sum(Preofc)|       sum(Preofv)|sum(Totneg)| sum(Quatot)|
+--------+------------+-----------+------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------+------------+
|20011127|        1210|24213463670|       62200|          29070| 50535.26000000001| 51128.23999999999| 49998.44000000003|50502.079999999965|          50624.08| 29438.13999999998|26229.700000000063|      76004|158022506246|
|20020325|         528|10570731600|       26684|          11814| 26158.22999999998|26322.389999999996| 25893

In [76]:
dfGbSum.rdd.getNumPartitions()

200

In [81]:
df.rdd.repartition(50)

MapPartitionsRDD[209] at coalesce at NativeMethodAccessorImpl.java:0

In [105]:
import pyspark
df.persist(pyspark.StorageLevel.MEMORY_ONLY)

22/08/13 11:16:25 WARN CacheManager: Asked to cache already cached data.


DataFrame[TypeReg: int, Date: int, BDICode: int, Codneg: string, MarketType: int, Company: string, Spec: string, Prazot: string, Currency: string, Open: double, Max: double, Min: double, Med: double, Close: double, Preofc: double, Preofv: double, Totneg: int, Quatot: bigint]

In [106]:
df.write.mode("overwrite").parquet("/home/jovyan/Bovespa.parquet")

22/08/13 11:24:57 WARN MemoryStore: Not enough space to cache rdd_220_6 in memory! (computed 48.2 MiB so far)
22/08/13 11:24:57 WARN MemoryStore: Not enough space to cache rdd_220_2 in memory! (computed 47.2 MiB so far)
22/08/13 11:24:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
